# Engenharia de Softwares Inteligentes - MVP - Titanic Challenge

Desafio, dados originais e detalhes da base disponiveis em:

https://www.kaggle.com/competitions/titanic

### Descrição
A base de dados utilizada é parte do _Titanic Challenge_, famoso desafio permanente do Kaggle.com para Machine Learning. O objetivo é que, a partir dos dados pessoais e de viagem de cada um dos passageiros, seja possível determinar a sobrevivência de cada um deles. Pela natureza subjetiva da escolha dos que teriam ou não acesso aos botes, não é possível estabelecer uma correlação plena, e valores muito elevados normalmente são sinais de que o teste está sobreajustado (overfitting). Contudo, acreditamos que há um padrão na escolha dos sobreviventes, aplicável na ampla maioria dos casos. Serão criadas classes para demonstrar o potencial da utilização de boas práticas em Engenharia de Software. As classes servirão também para tratar dados fornecidos pelo usuário, que serão testados contra o modelo.<br>
Colunas:
-  Survival: Se a pessoa sobreviveu. Será nosso target.
-  Pclass: [1,2,3] Primeira, Segunda e Terceira Classe
-  Sex: Sexo do Passageiro
-  Age: Idade
-  SibSP: Irmãos e Conjuges
-  Parch: Filhos ou Pais
-  Ticket: Contem codigos que nao consegui explicar a natureza
-  Fare: Valor do ingresso
-  Cabin: Número da Cabine
-  Embarked: Inicial do porto de embarque. <br>

## Etapa I - Carga e Processamento dos Dados

In [39]:
# Configurações do notebook dentre outros
import warnings
warnings.filterwarnings("ignore")

# Imports gerais
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Imports específicos do Scikit-lean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [40]:
# Arquivos do Kaggle
url = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/data/titanic/train.csv"

# Carga no Pandas
df_treino = pd.read_csv(url, delimiter=',')

In [41]:
df_treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Pronomes de Tratamento

A classe, título ou qualificação do passegeiro pode ter sido fator preponderante para determinar se uma pessoa foi ou não escolhida para ir aos botes.

In [42]:
def pega_pronome_tratamento(entrada):
    # A entrada deve ser um dataframe. Pega o Mr. Mrs. e coloca em coluna própria
    pronome_tratamento = re.search(r' ([A-Za-z]+)\.',entrada)
    if pronome_tratamento:
        return pronome_tratamento.group(1)
    return ""

df_treino['Pronome'] = df_treino['Name'].apply(pega_pronome_tratamento)

Vejamos qual a distribuição desses pronomes entre os sobreviventes.

In [43]:
grouped = df_treino.groupby(['Pronome', 'Survived']).size().unstack(fill_value=0)
distribuicao = grouped.div(grouped.sum(axis=1), axis=0) * 100
distribuicao.drop(0, axis=1, inplace=True)
distribuicao
# Valores serão apresentados como % do total

Survived,1
Pronome,
Capt,0.000000
Col,50.000000
Countess,100.000000
Don,0.000000
Dr,42.857143
Jonkheer,0.000000
Lady,100.000000
Major,50.000000
Master,57.500000


Classificarei os passageiros, através do seu pronome pela probabilidade de ser poupado ou de se sacrificar, com base na lista acima. 5 para chances altas de se sacrificar até 1, para chances altas de ser poupado.

In [44]:
estoicismo = {
    'Capt': 5,
    'Col': 3,
    'Countess': 1,
    'Don': 5,
    'Dr': 3,
    'Jonkheer': 5,
    'Lady': 1,
    'Major': 3,
    'Master': 3,
    'Miss': 4,
    'Mlle': 1,
    'Mme': 1,
    'Mr': 2,
    'Mrs': 2,
    'Ms': 1,
    'Rev': 5,
    'Sir': 1
}
df_treino['Aval_Pronome'] = df_treino["Pronome"].apply(lambda x: estoicismo.get(x))

## Porto de Embarque
A  compreensão do porto de embarque pode ser relevante uma vez que praticamente toda a tripulação partiu de Southampton.

In [45]:
# Transforma o código do porto de embarque em escalar.
le = LabelEncoder()
df_treino["Embarked"] = df_treino["Embarked"].apply(lambda x: 'A' if x == 28.0 else x ) # Tinha uma linha com float ao inves de string
df_treino["cod_embarque"] = le.fit_transform(df_treino['Embarked'])

## Valor do Ticket
Os valores de ticket estao errados pois correspondem provavelmente ao valor de face (observe que quem viaja com família tem sempre tickets maiores). Vamos somar os integrantes da família e dividir por cabeça o valor do ticket.

In [46]:
df_treino["FixedFare"] = df_treino["Fare"]/(df_treino["SibSp"] + df_treino["Parch"] + 1)
df_treino["FixedFare"] = df_treino["FixedFare"].apply(np.round)

## Categorias de Idade
Vamos criar categorias de idade para agrupar os passageiros no que consideramos ser categorias de idade relevantes para o processo de escolha dos sobreviventes. Contudo há algumas idades que estão em branco. Vamos preencher a idade faltante com a mediana da idade das pessoas com aquele pronome de tratamento. 

In [58]:
# Pega a média das idades por pronome
pronomes = df_treino["Pronome"].unique()

# Pega a mediana por pronome
mediana = {}
for pronome in pronomes:
    mediana[pronome] = df_treino["Age"][df_treino["Pronome"] == pronome].median()

# Preenche as idades em branco com a media das idades para aquele pronome
df_treino['Age'] = df_treino.apply(lambda x: mediana[x['Pronome']] if pd.isna(x['Age']) else x['Age'], axis=1)

# Converte para inteiro
df_treino["Age"] = df_treino["Age"].astype(int)

# Cria uma feature nova, categorizando os passageiros em idades relevantes para a decisão de acesso ao bote
df_treino["classe_idade"] = pd.cut(df_treino["Age"], bins=[-np.inf,5,10,13,60,np.inf], labels=["Colo","Nova","Jovem","Adulto", "Idoso"])

In [70]:
# Sobreviventes por classe de idade
grouped = df_treino.groupby(['classe_idade', 'Survived']).size().unstack(fill_value=0)
distribuicao = grouped.div(grouped.sum(axis=1), axis=0) * 100
distribuicao.drop(0, axis=1, inplace=True) # Não há necessidade da coluna de não-sobreviventes
distribuicao

Survived,1
classe_idade,
Colo,68.750000
Nova,35.000000
Jovem,57.142857
Adulto,36.901763
Idoso,22.727273
